<a href="https://colab.research.google.com/github/gabordun/web_scraping/blob/master/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
#import packages

import numpy as np
import pandas as pd
import csv
from bs4 import BeautifulSoup
import re

#import requests

from urllib import request
import requests
import time

#finding sources


url = 'http://www.ebizmba.com/articles/news-websites'

def getAllUrl(url):
    try:
        page = request.urlopen(url).read()
    except:
        return []
    urlList = []
    try:
        soup = BeautifulSoup(page)
        soup.prettify()
        for anchor in soup.findAll('a', href=True):
            if not 'http://' in anchor['href']:
                if urlparse.urljoin(url, anchor['href']) not in urlList:
                    urlList.append(urlparse.urljoin(url, anchor['href']))
            else:
                if anchor['href'] not in urlList:
                    urlList.append(anchor['href'])

        length = len(urlList)

        return urlList
    except request.HTTPError as e:
        print(e)

print(getAllUrl(url))


['http://www.ebizmba.com/', 'http://www.ebizmba.com/directory#admin', 'http://www.ebizmba.com/directory#design', 'http://www.ebizmba.com/directory#marketing', 'http://www.ebizmba.com/directory#ecommerce', 'http://www.ebizmba.com/directory', 'http://www.ebizmba.com/directory#media', 'http://www.ebizmba.com/directory#research', "javascript:bookmarksite('eBizMBA | The eBusiness Knowledgebase', 'http://www.ebizmba.com')", 'http://www.ebizmba.com/terms', 'http://www.ebizmba.com/privacy', 'http://news.yahoo.com', 'http://siteanalytics.compete.com/news.yahoo.com', 'http://quantcast.com/news.yahoo.com', 'http://www.alexa.com/siteinfo/news.yahoo.com/', 'http://news.google.com', 'http://siteanalytics.compete.com/news.google.com', 'http://quantcast.com/news.google.com', 'http://www.alexa.com/siteinfo/news.google.com/', 'http://www.huffingtonpost.com', 'http://siteanalytics.compete.com/huffingtonpost.com', 'http://quantcast.com/huffingtonpost.com', 'http://www.alexa.com/siteinfo/huffingtonpost.com

ValueError: ignored

In [148]:
# run counter

for url in getAllUrl(url):
  basic=requests.get(url).text
  soup=BeautifulSoup(basic, 'html.parser')

  #b = soup.prettify()
  #print(b)

  keywords={'corona','Corona','coronavirus','corona virus','Corona Virus', 'Coronavirus'}

  for x in keywords:
   x = soup(text=re.compile('x'))
   counter = 0
   for item in x:
    counter += 1
  
  print(counter)
  

18
42
42
42
42
42
42
42


InvalidSchema: ignored

In [0]:
seed_urls = ['https://welt.de',
             'https://nytimes.com',
             'https://theguardian.com',
             'https://lefigaro.fr',
             'https://elpais.com']

for url in seed_urls:
  basic=requests.get(url).text
  soup=BeautifulSoup(basic, 'html.parser')

  #b = soup.prettify()
  #print(b)

  keywords={'corona','Corona','coronavirus','corona virus','Corona Virus', 'Coronavirus'}

  for x in keywords:
   x = soup(text=re.compile('x'))
   rowcounter = 0
   for item in x:
    rowcounter += 1
  
  y = soup.find_all('a')
  colcounter = 0
  for item in y:
    colcounter += 1

  print(rowcounter)
  print(colcounter)

  result = []
  for i in range(0,colcounter):
    a=y[i].get_text("title").replace("title"," ").replace("\n"," ")
    result.append(a)
 
  df = pd.DataFrame(data = result)
  print(df)

#df.to_csv("formula1.csv")
#uploaded = drive.CreateFile({'title': 'formula1.csv'})
#uploaded.SetContentFile("formula1.csv")
#uploaded.Upload()